## 1. Setup and Initialization

In [7]:
# Install the required libraries
!pip install groq panel

Defaulting to user installation because normal site-packages is not writeable


In [8]:
# Import necessary libraries
from groq import Groq
import panel as pn

# Initialize the Panel extension to enable interactive components
pn.extension()

In [9]:
# --- Configuration ---

# IMPORTANT: Paste your Groq API key here.
# Get a free key from https://console.groq.com/keys

API_KEY = ""

# Initialize the Groq client
client = Groq(api_key=API_KEY)

## 2. Core Functions

In [10]:
def get_completion_from_messages(messages, model="llama3-8b-8192", temperature=0):
    """
    Sends a list of messages to the Groq API and returns the model's response.
    
    Args:
        messages (list): A list of message dictionaries.
        model (str): The name of the model to use.
        temperature (float): Controls the randomness of the output.
        
    Returns:
        str: The content of the assistant's message.
    """
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=temperature,
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"An error occurred: {e}")
        return "Sorry, there was an error connecting to the service."

## 3. Chatbot Configuration and UI

In [11]:
# This list will store the conversation history.
panels = [] 

# The system prompt defines the bot's behavior and menu.
context = [ {'role':'system', 'content':"""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a very short, very conversational friendly style. \
Don't forgot to put prices for the ordered items, while responding. \
If you made any mistake, offer a discount of 1.00 and adjust the total based on discounted amount. \
Always show the items in bulleted list with prices.
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""} ]

In [12]:
# Create the UI widgets
inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…', width=600)
button_conversation = pn.widgets.Button(name="Chat!", button_type='primary')

def collect_messages(_):
    """
    This function is called when the 'Chat!' button is clicked.
    It collects the user's prompt, gets the bot's response, and updates the UI.
    """
    prompt = inp.value
    inp.value = ''
    
    # Add user message to context and display it
    context.append({'role':'user', 'content':f"{prompt}"})
    panels.append(
        pn.Row('**You:**', pn.pane.Markdown(prompt, width=600)))
    
    # Get bot response and display it
    response = get_completion_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('**Bot:**', pn.pane.Markdown(response, width=600, styles={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)

# Bind the function to the button
interactive_conversation = pn.bind(collect_messages, button_conversation)

# Create the final dashboard layout
dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=400, scroll=True),
)

# Display the dashboard. This will start the interactive chat.
dashboard

Column
    [0] TextInput(placeholder='Enter text here…', width=600)
    [1] Row
        [0] Button(button_type='primary', name='Chat!')
    [2] ParamFunction(function, _pane=Column, defer_load=False, height=400, loading_indicator=True)

## 4. Summarize the Final Order

In [13]:
# Create a copy of the conversation history
messages =  context.copy()

# Add a new user message asking for a summary
messages.append(
    {'role':'user', 'content':'Create a Markdown summary of the previous food order. Itemize the price for each item. The fields should be 1) pizza, include size 2) list of toppings 3) list of drinks, include size 4) list of sides include size 5) total price'},
)

# Get the summary from the model
response = get_completion_from_messages(messages, temperature=0)

# Print the response
print(response)

**Food Order Summary**

### Pizza
* **Medium Cheese Pizza**: $9.25
	+ Toppings: Mushrooms ($1.50), Green Peppers ($1.00)

### Drinks
* **Small Coke**: $2.00

### Sides
* **Small Fries**: $3.50

### Total Price
* **$16.25**

Note: The total price does not include the delivery fee of $2.00, which was mentioned earlier.


## 5. Clear Conversation History (Optional)
If you want to start a new order and clear the bot's memory, run the following code.

In [14]:
# Uncomment the line below to delete the conversation history
del(context, messages)